# Importing all the good stuff

In [1]:
import numpy as np
import pandapower as pp
import pandapower.networks as pn
import pandapower.converter as pc

net = pp.networks.case4gs() #Load a case 

# Ybus: Pi model with pandapower

In [2]:
ppc = pc.to_ppc(net)
Ybus, Yf, Yt = pp.makeYbus_pypower(ppc["baseMVA"], ppc["bus"], ppc["branch"])
print(Ybus/100)

  (0, 0)	(8.98519043680334-44.835952184016705j)
  (2, 0)	(-5.169561621174524+25.847808105872623j)
  (1, 0)	(-3.815628815628816+19.07814407814408j)
  (1, 1)	(8.98519043680334-44.835952184016705j)
  (0, 1)	(-3.815628815628816+19.07814407814408j)
  (3, 1)	(-5.169561621174524+25.847808105872623j)
  (2, 2)	(8.193267475069058-40.86383737534529j)
  (0, 2)	(-5.169561621174524+25.847808105872623j)
  (3, 2)	(-3.023705853894534+15.118529269472667j)
  (2, 3)	(-3.023705853894534+15.118529269472667j)
  (1, 3)	(-5.169561621174524+25.847808105872623j)
  (3, 3)	(8.193267475069058-40.86383737534529j)


# Ybus: Simple model 

In [3]:
# Extract needed information to build Ybus matrix

nbus = len(net.bus) # Number of buses in the system
r = net.line["length_km"]*net.line["r_ohm_per_km"] # Total resistance of lines
x = net.line["length_km"]*net.line["x_ohm_per_km"] # Total reactance of lines
z = r.values+x.values*1j # Total resistance of lines
from_buses = net.line["from_bus"].values # "From bus" vector
to_buses = net.line["to_bus"].values  # "To bus" vector
Zb=(135000**2)/(100*10**6)
Zpu = z/Zb

#
# Calculating Yij and Yjj elements seperately
#

# From definition:  the off diagonal entries are the negatives of the admittance
#                   of lines between buses i and j. If there is no line between i
#                   and j this term is zero
yij = 1/Zpu


# From definition:  the diagonal entries of Y bus are found by summing the primitive 
#                   admittance of lines lines and ties to the reference at bus j.

bus_connections = np.concatenate([from_buses, to_buses])
y_connections = np.concatenate([yij,yij])
yjj = np.zeros(nbus,dtype=np.complex_)
for i in range(0,nbus):
    yjj[i] = sum(y_connections[np.where((bus_connections == i))])

Ybus = np.zeros((nbus,nbus),dtype=np.complex_)
for i in range(0,nbus):
    Ybus[from_buses[i]][to_buses[i]] = -yij[i]
    Ybus[to_buses[i]][from_buses[i]] = -yij[i]
    Ybus[i][i]                       = yjj[i]

In [4]:
Ybus

array([[ 3.09555947-15.47779733j, -1.31455265 +6.57276325j,
        -1.78100682 +8.90503408j,  0.         +0.j        ],
       [-1.31455265 +6.57276325j,  3.09555947-15.47779733j,
         0.         +0.j        , -1.78100682 +8.90503408j],
       [-1.78100682 +8.90503408j,  0.         +0.j        ,
         2.82272778-14.11363892j, -1.04172097 +5.20860484j],
       [ 0.         +0.j        , -1.78100682 +8.90503408j,
        -1.04172097 +5.20860484j,  2.82272778-14.11363892j]])